<a href="https://colab.research.google.com/github/jespimentel/sis_geo/blob/main/plotador_trafico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import folium, random
from folium.plugins import HeatMap, TagFilterButton

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/registros_nos_limites_0_2000.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/registros_nos_limites_2001_3314.csv')
df_partes = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/partes.csv')
df = pd.concat([df_1,df_2] , ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Número TJ             3168 non-null   object 
 1   Número MP             3168 non-null   object 
 2   Tipo de Procedimento  3168 non-null   object 
 3   Situação              3168 non-null   object 
 4   Delegacia             3164 non-null   object 
 5   Assunto               3168 non-null   object 
 6   DtFatoInicial         3168 non-null   object 
 7   Violência Doméstica   3167 non-null   object 
 8   Município             3168 non-null   object 
 9   Logradouro            3168 non-null   object 
 10  Num_Logradouro        2904 non-null   object 
 11  Bairro                3019 non-null   object 
 12  Latitude              3168 non-null   float64
 13  Longitude             3168 non-null   float64
dtypes: float64(2), object(12)
memory usage: 346.6+ KB


In [ ]:
print(df_partes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3592 entries, 0 to 3591
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Número MP  3592 non-null   object
 1   Partes     3592 non-null   object
dtypes: object(2)
memory usage: 56.2+ KB
None


In [ ]:
df['Assunto'].value_counts().head(10)

 Ameaça                                764
 Tráfico de Drogas e Condutas Afins    593
 Contra a Mulher                       193
 Furto Qualificado                     185
 Vias de fato                          183
 Furto                                 138
 Injúria                               130
 Homicídio Qualificado                  91
 Estelionato                            59
 Estupro de vulnerável                  57
Name: Assunto, dtype: int64

In [ ]:
# Criando novo dataframe com o assunto que interessa

criterio = (df['Assunto'] == ' Tráfico de Drogas e Condutas Afins') & (df['Situação'] == 'DENUNCIADO')
df_filtrado = df.loc[criterio][['Número MP', 'Número TJ', 'Assunto', 'DtFatoInicial','Latitude','Longitude']]
df_filtrado['Assunto'] = df_filtrado['Assunto'].apply(lambda x: x.strip())
df_filtrado

,Número MP,Número TJ,Assunto,DtFatoInicial,Latitude,Longitude
7,13.0723.0002119/2023-7,1500815-66.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,2023-04-13,-22.74584,-47.67720
10,13.0723.0002017/2023-0,1500799-15.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,2023-04-11,-22.67482,-47.69167
23,13.0723.0002094/2023-5,1500780-09.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,2023-04-06,-22.67035,-47.65785
42,24.0723.0001815/2023-9,1500730-80.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,2023-03-30,-22.76780,-47.62849
62,13.0723.0001817/2023-8,1500714-29.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,2023-03-27,-22.75088,-47.65960
...,...,...,...,...,...,...
3127,13.0723.0000083/2022-4,1500045-10.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,2022-01-05,-22.75249,-47.63069
3147,13.0723.0000219/2022-2,1500018-27.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,2022-01-03,-22.75581,-47.65983
3149,13.0723.0000560/2022-4,1500014-87.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,2022-01-03,-22.69542,-47.66435
3150,13.0723.0000447/2022-1,1500016-57.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,2022-01-03,-22.67422,-47.69469


In [ ]:
# Converter a coluna 'DtFatoInicial' para o tipo datetime
df_filtrado['DtFatoInicial'] = pd.to_datetime(df_filtrado['DtFatoInicial'], dayfirst=False)
df_filtrado['DiaDaSemana'] = df_filtrado['DtFatoInicial'].dt.day_name()
df_filtrado['DtFatoInicial'] = df_filtrado['DtFatoInicial'].dt.strftime('%d/%m/%Y')

traducao_dias_semana = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df_filtrado['DiaDaSemana'] = df_filtrado['DiaDaSemana'].map(traducao_dias_semana)

df_filtrado

,Número MP,Número TJ,Assunto,DtFatoInicial,Latitude,Longitude,DiaDaSemana
7,13.0723.0002119/2023-7,1500815-66.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,13/04/2023,-22.74584,-47.67720,Quinta-feira
10,13.0723.0002017/2023-0,1500799-15.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,11/04/2023,-22.67482,-47.69167,Terça-feira
23,13.0723.0002094/2023-5,1500780-09.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,06/04/2023,-22.67035,-47.65785,Quinta-feira
42,24.0723.0001815/2023-9,1500730-80.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,30/03/2023,-22.76780,-47.62849,Quinta-feira
62,13.0723.0001817/2023-8,1500714-29.2023.8.26.0599,Tráfico de Drogas e Condutas Afins,27/03/2023,-22.75088,-47.65960,Segunda-feira
...,...,...,...,...,...,...,...
3127,13.0723.0000083/2022-4,1500045-10.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,05/01/2022,-22.75249,-47.63069,Quarta-feira
3147,13.0723.0000219/2022-2,1500018-27.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,03/01/2022,-22.75581,-47.65983,Segunda-feira
3149,13.0723.0000560/2022-4,1500014-87.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,03/01/2022,-22.69542,-47.66435,Segunda-feira
3150,13.0723.0000447/2022-1,1500016-57.2022.8.26.0599,Tráfico de Drogas e Condutas Afins,03/01/2022,-22.67422,-47.69469,Segunda-feira


In [ ]:
# Mapa de calor

# Extrai as colunas de latitude e longitude do dataset
latitude = df_filtrado['Latitude']
longitude = df_filtrado['Longitude']

PIRA_COORDINATES = (-22.769960, -47.647579)

# Cria um mapa vazio em Piracicaba
mapa = folium.Map(location=PIRA_COORDINATES, zoom_start=12)

# Agrupa as coordenadas de latitude e longitude em uma lista
dados = list(zip(latitude, longitude))

# Adiciona o mapa de calor
heatmap = HeatMap(dados).add_to(mapa)

# Adiciona um título ao mapa
title_html = """
<h1 style="text-align:center;"><b>Mapa de calor do tráfico - denunciados para data do fato entre 01/01/2022 e 24/04/2023</b></h1>
<h2 style="text-align:center;"><b>Base: SISMP INTEGRADO - Extração em 24/04/2023</b></h2>
"""
mapa.get_root().html.add_child(folium.Element(title_html))

folium.TileLayer('openstreetmap').add_to(mapa)
folium.LayerControl().add_to(mapa)

# Salva o mapa em um arquivo HTML
mapa.save("heatmap_trafico.html")
mapa

In [ ]:
merged_df = pd.merge(df_filtrado, df_partes, on= 'Número MP')
merged_df.reset_index(drop=True, inplace=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Número MP      606 non-null    object 
 1   Número TJ      606 non-null    object 
 2   Assunto        606 non-null    object 
 3   DtFatoInicial  606 non-null    object 
 4   Latitude       606 non-null    float64
 5   Longitude      606 non-null    float64
 6   DiaDaSemana    606 non-null    object 
 7   Partes         606 non-null    object 
dtypes: float64(2), object(6)
memory usage: 38.0+ KB


In [ ]:
partes = list(set(merged_df['Partes'].tolist()))
dias_da_semana = list(set(df_filtrado['DiaDaSemana'].tolist()))
partes [:10]

In [ ]:
# Locais da prisão

PIRA_COORDINATES = (-22.769960, -47.647579)

# Cria um mapa vazio em Piracicaba
_map = folium.Map(location=PIRA_COORDINATES, zoom_start=12)

# Adiciona um título ao mapa
title_html = """
<h1 style="text-align:center;"><b>Tráficos denunciados para data do fato entre 01/01/2022 e 24/04/2023</b></h1>
<h2 style="text-align:center;"><b>Base: SISMP INTEGRADO - Extração em 24/04/2023</b></h2>
"""
_map.get_root().html.add_child(folium.Element(title_html))

# Adiciona o marcador para cada ocorrência
for each in merged_df.iterrows():
    parte = each[1]['Partes']
    dia_da_semana = each[1]['DiaDaSemana']
    folium.Marker(
        location=[each[1]["Latitude"], each[1]["Longitude"]],
        clustered_marker=True,
        tags=[dia_da_semana],
        popup=f"{parte}<br>{each[1]['DtFatoInicial']}<br>{each[1]['Número TJ']}<br>{each[1]['DiaDaSemana']}",
    ).add_to(_map)

# Adiciona botões de filtro por tag
TagFilterButton(dias_da_semana, clear_text="Todos",name='DiaDaSemana').add_to(_map)

display(_map)
_map.save('traficantes.html')